<a href="https://colab.research.google.com/github/pwesquire/mb_colab/blob/main/Profanity_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Run on TensorFlow 2.x
%tensorflow_version 2.x

import numpy as np
import pandas as pd
import tensorflow as tf
# !pip install better_profanity
# from better_profanity import profanity as pf 
from pathlib import Path
!pip install -q -U "tensorflow-text==2.8.*"
import tensorflow_text as tft
import tensorflow_hub as hub
!pip install -q tf-models-official==2.7.0
from official.nlp import optimization 
# import functools
import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

bad_names = pd.read_csv("/content/drive/MyDrive/formatted_bad_names.csv")
good_names = pd.read_csv("/content/drive/MyDrive/international_names.csv")

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
     |████████████████████████████████| 1.8 MB 5.6 MB/s 
     |████████████████████████████████| 238 kB 55.2 MB/s 
     |████████████████████████████████| 116 kB 52.7 MB/s 
     |████████████████████████████████| 99 kB 8.2 MB/s 
     |████████████████████████████████| 1.3 MB 56.4 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 1.1 MB 48.7 MB/s 
     |████████████████████████████████| 352 kB 42.3 MB/s 


In [ ]:
rand_good = good_names.sample(2224, ignore_index=True)
bad_names.fillna(rand_good, inplace=True)
bad_names_titlecase = bad_names.transform({"first_name": lambda x: x.title(), "last_name": lambda x: x.title(), "gender": lambda x: x, "country_code": lambda x: x})
print(bad_names_titlecase)

     first_name    last_name gender country_code
0            A.      Blinkin      F           PR
1            A.     Blyncoln      F           TW
2       A. Cora     Spondent      M           DE
3     A. Nathan    Undevided    NaN           BE
4            A.    Nellsechs      M           HR
...         ...          ...    ...          ...
2219       Zack    Religious      M           NO
2220    Zeke N.  Yeshallfind      M           KR
2221      Zelda         Kowz      M           IL
2222        Zig     Steenine      F           BO
2223     Zoltan       Pepper      F           HN

[2224 rows x 4 columns]


In [ ]:
good_names = good_names.assign(label=0.0)
bad_names = bad_names_titlecase.assign(label=1.0)

In [ ]:
all_names = good_names.append(bad_names, ignore_index=True)

all_names.fillna({"first_name": "", "last_name": "", "country_code": "NA"})
all_names['full_name'] = all_names[['first_name','last_name']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

def label_profanity(name):
  if pf.contains_profanity(name):
    return 1.0
  else:
    return 0.0

# all_names['label'] = all_names[['full_name']].apply(lambda row: label_profanity(row['full_name']), axis=1)
all_names = all_names.drop(["first_name","last_name","gender"], axis=1)
all_names_rand = all_names.reindex(np.random.permutation(all_names.index))

filepath = Path('/content/drive/MyDrive/labeled_names.csv') 
all_names_rand.to_csv(filepath, index=False)  

In [ ]:
train_df = all_names_rand.head(40000)
val_df = all_names_rand.iloc[45001:50000]
test_df = all_names_rand.tail(5000)

In [ ]:
train_df = train_df.astype({'full_name': 'string', 'country_code': 'string', 'label':'float32'})
val_df = val_df.astype({'full_name': 'string', 'country_code': 'string', 'label':'float32'})
test_df = test_df.astype({'full_name': 'string', 'country_code': 'string', 'label':'float32'})
# print(train_df['full_name'].dtype)
# print(all_names.tail(5))

In [ ]:
train_labels = train_df.pop('label')
val_labels = val_df.pop('label')
test_labels = test_df.pop('label')
train_ds = tf.data.Dataset.from_tensor_slices((tf.constant(train_df['full_name'],), train_labels))
val_ds = tf.data.Dataset.from_tensor_slices((tf.constant(val_df['full_name'],), val_labels))
test_ds = tf.data.Dataset.from_tensor_slices((tf.constant(test_df['full_name'],), test_labels))


AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'

In [ ]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [ ]:
classifier_model = build_classifier_model()

epochs = 5
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [ ]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(x=train_ds,
                               validation_data=val_ds,
                               epochs=epochs)

Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Epoch 1/5


ValueError: ignored

In [ ]:
loss, accuracy = classifier_model.evaluate(test_ds)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

In [ ]:

# train_labels = tf.constant(train_df['label'])

# Preprocess the input strings.
train_data = preprocess_data(train_ds)
val_data = preprocess_data(val_ds)
test_data = preprocess_data(test_ds)

# Build the Keras model.
keras_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=[None], dtype=tf.int64),

    # Feature layer goes here?
    tf.keras.layers.Dense(32),
    tf.keras.layers.Activation(tf.nn.relu),
    tf.keras.layers.Dense(1)
])

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
keras_model.compile(loss='binary_crossentropy', optimizer='rmsprop',
              metrics=['accuracy'])
keras_model.fit(train_data, train_labels, epochs=5,callbacks=[tensorboard_callback])
# print(keras_model.predict(test_data))

ValueError: ignored

In [ ]:
test_bad = pd.DataFrame(data={'full_name': "BJ CUMMINGS", 'country_code': 'CA'}, index=[0]).astype({'full_name': 'string', 'country_code': 'string'})
test_good = pd.DataFrame(data={'full_name': "Bjork Gonzo", 'country_code': 'JP'}, index=[0]).astype({'full_name': 'string', 'country_code': 'string'})
processed_bad = preprocess_data(test_bad)
processed_good = preprocess_data(test_good)

test_prediction_bad = keras_model.predict(processed_bad)
test_prediction_good = keras_model.predict(processed_good)

print("bad name prediction:",test_prediction_bad)

print("good name prediction:",test_prediction_good)

bad name prediction: [[-0.01348143]]
good name prediction: [[-0.01752897]]
